In [1]:
# These are standard python modules
import json, time, urllib.parse

# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

# The 'pandas' module is for reading and processing csv files
import pandas as pd

### Import US Cities List

The list of articles of cities in the US is scraped from Wikipedia and saved as a .csv file. In this table, there are 22,157 cities in total.

In [2]:
us_city_state = pd.read_csv("us_cities_by_state_SEPT.2023.csv")
print(us_city_state.shape)
us_city_state.head()

(22157, 3)


,state,page_title,url
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama"
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama"
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama"
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama"
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama"


After checking the duplicates in this table, the total number of of cities is lowered to 21,525.

In [3]:
us_city_state.drop_duplicates(inplace=True)
print(us_city_state.shape)
us_city_state.head()

(21525, 3)


,state,page_title,url
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama"
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama"
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama"
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama"
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama"


To make request for every article in the list, here is a list of the US city article titles:

In [4]:
us_city_titles = us_city_state.page_title
us_city_titles

0         Abbeville, Alabama
1        Adamsville, Alabama
2           Addison, Alabama
3             Akron, Alabama
4         Alabaster, Alabama
                ...         
22152     Wamsutter, Wyoming
22153     Wheatland, Wyoming
22154       Worland, Wyoming
22155        Wright, Wyoming
22156         Yoder, Wyoming
Name: page_title, Length: 21525, dtype: object

### Make Page Info Request

To make requests for every article page, here are some constants defined for using the API. The API endpoint, request headers, and page info parameters template are defined.

In [5]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<gjwong@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = us_city_titles

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


The following functions is defined to make 1 request for 1 article. The input required for the function is a string of article title.

In [6]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


With a for loop over all the article titles in my list, the title and its corresponding revision id are collected and stored in 2 separate lists. It's worth noticed that this process is taking a significantly long time.

In [7]:
titles, revIds = [], []

for article in ARTICLE_TITLES:
    print(f"Getting page info data for: {article}")
    info = request_pageinfo_per_article(article)
    for pageid, page_info in info['query']['pages'].items():
        title = page_info['title']
        lastrevid = page_info['lastrevid']
        
        titles.append(title)
        revIds.append(lastrevid)

Getting page info data for: Abbeville, Alabama
Getting page info data for: Adamsville, Alabama
Getting page info data for: Addison, Alabama
Getting page info data for: Akron, Alabama
Getting page info data for: Alabaster, Alabama
Getting page info data for: Albertville, Alabama
Getting page info data for: Alexander City, Alabama
Getting page info data for: Aliceville, Alabama
Getting page info data for: Allgood, Alabama
Getting page info data for: Altoona, Alabama
Getting page info data for: Andalusia, Alabama
Getting page info data for: Anderson, Lauderdale County, Alabama
Getting page info data for: Anniston, Alabama
Getting page info data for: Arab, Alabama
Getting page info data for: Ardmore, Alabama
Getting page info data for: Argo, Alabama
Getting page info data for: Ariton, Alabama
Getting page info data for: Arley, Alabama
Getting page info data for: Ashford, Alabama
Getting page info data for: Ashland, Alabama
Getting page info data for: Ashville, Alabama
Getting page info dat

Getting page info data for: Graysville, Alabama
Getting page info data for: Greensboro, Alabama
Getting page info data for: Greenville, Alabama
Getting page info data for: Grimes, Alabama
Getting page info data for: Grove Hill, Alabama
Getting page info data for: Guin, Alabama
Getting page info data for: Gulf Shores, Alabama
Getting page info data for: Guntersville, Alabama
Getting page info data for: Gurley, Alabama
Getting page info data for: Gu-Win, Alabama
Getting page info data for: Hackleburg, Alabama
Getting page info data for: Haleburg, Alabama
Getting page info data for: Haleyville, Alabama
Getting page info data for: Hamilton, Alabama
Getting page info data for: Hammondville, Alabama
Getting page info data for: Hanceville, Alabama
Getting page info data for: Harpersville, Alabama
Getting page info data for: Hartford, Alabama
Getting page info data for: Hartselle, Alabama
Getting page info data for: Hayden, Alabama
Getting page info data for: Hayneville, Alabama
Getting page i

Getting page info data for: Red Bay, Alabama
Getting page info data for: Red Level, Alabama
Getting page info data for: Reece City, Alabama
Getting page info data for: Reform, Alabama
Getting page info data for: Rehobeth, Alabama
Getting page info data for: Repton, Alabama
Getting page info data for: Ridgeville, Alabama
Getting page info data for: River Falls, Alabama
Getting page info data for: Riverside, Alabama
Getting page info data for: Riverview, Alabama
Getting page info data for: Roanoke, Alabama
Getting page info data for: Robertsdale, Alabama
Getting page info data for: Rockford, Alabama
Getting page info data for: Rogersville, Alabama
Getting page info data for: Rosa, Alabama
Getting page info data for: Russellville, Alabama
Getting page info data for: Rutledge, Alabama
Getting page info data for: St. Florian, Alabama
Getting page info data for: Samson, Alabama
Getting page info data for: Sand Rock, Alabama
Getting page info data for: Sanford, Alabama
Getting page info data 

Getting page info data for: Kotzebue, Alaska
Getting page info data for: Koyuk, Alaska
Getting page info data for: Koyukuk, Alaska
Getting page info data for: Kupreanof, Alaska
Getting page info data for: Kwethluk, Alaska
Getting page info data for: Larsen Bay, Alaska
Getting page info data for: Lower Kalskag, Alaska
Getting page info data for: Manokotak, Alaska
Getting page info data for: Marshall, Alaska
Getting page info data for: McGrath, Alaska
Getting page info data for: Mekoryuk, Alaska
Getting page info data for: Mountain Village, Alaska
Getting page info data for: Napakiak, Alaska
Getting page info data for: Napaskiak, Alaska
Getting page info data for: Nenana, Alaska
Getting page info data for: New Stuyahok, Alaska
Getting page info data for: Newhalen, Alaska
Getting page info data for: Nightmute, Alaska
Getting page info data for: Nikolai, Alaska
Getting page info data for: Nome, Alaska
Getting page info data for: Nondalton, Alaska
Getting page info data for: Noorvik, Alaska

Getting page info data for: Amity, Arkansas
Getting page info data for: Anthonyville, Arkansas
Getting page info data for: Antoine, Arkansas
Getting page info data for: Arkadelphia, Arkansas
Getting page info data for: Arkansas City, Arkansas
Getting page info data for: Ash Flat, Arkansas
Getting page info data for: Ashdown, Arkansas
Getting page info data for: Atkins, Arkansas
Getting page info data for: Aubrey, Arkansas
Getting page info data for: Augusta, Arkansas
Getting page info data for: Austin, Arkansas
Getting page info data for: Avoca, Arkansas
Getting page info data for: Bald Knob, Arkansas
Getting page info data for: Banks, Arkansas
Getting page info data for: Barling, Arkansas
Getting page info data for: Bassett, Arkansas
Getting page info data for: Batesville, Arkansas
Getting page info data for: Bauxite, Arkansas
Getting page info data for: Bay, Arkansas
Getting page info data for: Bearden, Arkansas
Getting page info data for: Beaver, Arkansas
Getting page info data for:

...

Getting page info data for: Mukilteo, Washington
Getting page info data for: Napavine, Washington
Getting page info data for: Newcastle, Washington
Getting page info data for: Newport, Washington
Getting page info data for: Nooksack, Washington
Getting page info data for: Normandy Park, Washington
Getting page info data for: North Bend, Washington
Getting page info data for: North Bonneville, Washington
Getting page info data for: Oak Harbor, Washington
Getting page info data for: Oakville, Washington
Getting page info data for: Ocean Shores, Washington
Getting page info data for: Okanogan, Washington
Getting page info data for: Olympia, Washington
Getting page info data for: Omak, Washington
Getting page info data for: Oroville, Washington
Getting page info data for: Orting, Washington
Getting page info data for: Othello, Washington
Getting page info data for: Pacific, Washington
Getting page info data for: Palouse, Washington
Getting page info data for: Pasco, Washington
Getting page

Getting page info data for: Morgantown, West Virginia
Getting page info data for: Wheeling, West Virginia
Getting page info data for: Martinsburg, West Virginia
Getting page info data for: Weirton, West Virginia
Getting page info data for: Fairmont, West Virginia
Getting page info data for: Beckley, West Virginia
Getting page info data for: Clarksburg, West Virginia
Getting page info data for: South Charleston, West Virginia
Getting page info data for: St. Albans, West Virginia
Getting page info data for: Vienna, West Virginia
Getting page info data for: Bluefield, West Virginia
Getting page info data for: Bridgeport, West Virginia
Getting page info data for: Oak Hill, West Virginia
Getting page info data for: Moundsville, West Virginia
Getting page info data for: Dunbar, West Virginia
Getting page info data for: Hurricane, West Virginia
Getting page info data for: Elkins, West Virginia
Getting page info data for: Charles Town, West Virginia
Getting page info data for: Nitro, West Virg

Getting page info data for: Capon Bridge, West Virginia
Getting page info data for: Union, West Virginia
Getting page info data for: Delbarton, West Virginia
Getting page info data for: Paw Paw, West Virginia
Getting page info data for: Jane Lew, West Virginia
Getting page info data for: Farmington, West Virginia
Getting page info data for: Matewan, West Virginia
Getting page info data for: Burnsville, West Virginia
Getting page info data for: West Logan, West Virginia
Getting page info data for: Bancroft, West Virginia
Getting page info data for: Clay, West Virginia
Getting page info data for: Junior, West Virginia
Getting page info data for: Fairview, West Virginia
Getting page info data for: Lost Creek, West Virginia
Getting page info data for: Windsor Heights, West Virginia
Getting page info data for: Pine Grove, West Virginia
Getting page info data for: Whitesville, West Virginia
Getting page info data for: Lester, West Virginia
Getting page info data for: Gilbert, West Virginia
G

Getting page info data for: Manitowoc, Wisconsin
Getting page info data for: Marinette, Wisconsin
Getting page info data for: Marion, Wisconsin
Getting page info data for: Markesan, Wisconsin
Getting page info data for: Marshfield, Wisconsin
Getting page info data for: Mauston, Wisconsin
Getting page info data for: Mayville, Wisconsin
Getting page info data for: Medford, Wisconsin
Getting page info data for: Mellen, Wisconsin
Getting page info data for: Menasha, Wisconsin
Getting page info data for: Menomonie, Wisconsin
Getting page info data for: Mequon, Wisconsin
Getting page info data for: Merrill, Wisconsin
Getting page info data for: Middleton, Wisconsin
Getting page info data for: Milton, Wisconsin
Getting page info data for: Milwaukee
Getting page info data for: Mineral Point, Wisconsin
Getting page info data for: Mondovi, Wisconsin
Getting page info data for: Monona, Wisconsin
Getting page info data for: Monroe, Wisconsin
Getting page info data for: Montello, Wisconsin
Getting 

Getting page info data for: Pine Bluffs, Wyoming
Getting page info data for: Pinedale, Wyoming
Getting page info data for: Pine Haven, Wyoming
Getting page info data for: Powell, Wyoming
Getting page info data for: Ranchester, Wyoming
Getting page info data for: Rawlins, Wyoming
Getting page info data for: Riverside, Wyoming
Getting page info data for: Riverton, Wyoming
Getting page info data for: Rock River, Wyoming
Getting page info data for: Rock Springs, Wyoming
Getting page info data for: Rolling Hills, Wyoming
Getting page info data for: Saratoga, Wyoming
Getting page info data for: Sheridan, Wyoming
Getting page info data for: Shoshoni, Wyoming
Getting page info data for: Sinclair, Wyoming
Getting page info data for: Star Valley Ranch, Wyoming
Getting page info data for: Sundance, Wyoming
Getting page info data for: Superior, Wyoming
Getting page info data for: Ten Sleep, Wyoming
Getting page info data for: Thayne, Wyoming
Getting page info data for: Thermopolis, Wyoming
Getting

Concatenate 2 lists together, here is a dataframe with 2 columns, the page title and article revision id:

In [8]:
df = pd.DataFrame(list(zip(titles, revIds)), columns =['page_title', 'article_revid']) 
df

,page_title,article_revid
0,"Abbeville, Alabama",1171163550
1,"Adamsville, Alabama",1177621427
2,"Addison, Alabama",1168359898
3,"Akron, Alabama",1165909508
4,"Alabaster, Alabama",1179139816
...,...,...
21520,"Wamsutter, Wyoming",1169591845
21521,"Wheatland, Wyoming",1176370621
21522,"Worland, Wyoming",1166347917
21523,"Wright, Wyoming",1166334449


Merging the dataframe above with the original US city list on the common field `page_title`, here is a complete table with the article revision id.

In [14]:
us_city_state_id = pd.merge(us_city_state, df, on="page_title").drop_duplicates()
us_city_state_id

,state,page_title,url,article_revid
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama",1171163550
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama",1177621427
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama",1168359898
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama",1165909508
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama",1179139816
...,...,...,...,...
21538,Wyoming,"Wamsutter, Wyoming","https://en.wikipedia.org/wiki/Wamsutter,_Wyoming",1169591845
21539,Wyoming,"Wheatland, Wyoming","https://en.wikipedia.org/wiki/Wheatland,_Wyoming",1176370621
21540,Wyoming,"Worland, Wyoming","https://en.wikipedia.org/wiki/Worland,_Wyoming",1166347917
21541,Wyoming,"Wright, Wyoming","https://en.wikipedia.org/wiki/Wright,_Wyoming",1166334449


Save this dataframe as a .csv file for reuse:

In [15]:
us_city_state_id.to_csv("us_cities_by_state_SEPT.2023_revid.csv", index=False)